# Final Project
Kalyn Howes & Chloe VanCory

COSC 311

12.15.21

Showing the effectiveness of several machine-learning algorithms on our Spotify data.

In [30]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import copy
import tqdm

### Class knnalg
Implementing the k nearest neighbors algorithm.

In [2]:
# Class with internal parameter k and methods train and predict
class knnalg:
    def __init__(self, k):
        self.k = k
        
    # i.The train method should take in an array of pre-labeled data and store them to be used in the kNN calculation later.
    # training the algorithm based on already found data
    def train(self, data, labels):
        self.data = data
        self.labels = labels
        
    # ii. The predict method should take in a single data point (of the same dimension as those used in training) 
    # and return the label of the kNN decision. Keep in mind that, in general, you may have to choose between multiple labels. 
    # In this case, ties are possible, but you can just decide to break them randomly or increase k to get more votes
    # send in a test point for it to be classified
    def predict(self, x):
        # need to find k closest neighbors to the point w the Euclidean norm
        # pair up each distance with the label of its point
        distances = [ (norm(x - self.data[i]), self.labels[i]) for i in range(len(self.data))]
        #print('Distances: ', distances)
        
        # order all the distances
        k_nearest = sorted(distances)
        #print('k_nearest: ', distances)
        
        # determine the labels of the top k points & do majority vote
        return max(Counter([p[1] for p in k_nearest[:k]]))

Helper Functions:

divideSets: Splits a given data set (randomly) into testing and training subsets.

norm: Finds the Euclidean norm of a set of points.

In [3]:
def divideSets(p,data):
    copydata = copy.deepcopy(data)
    # print(copydata)
    np.random.shuffle(copydata)
    list1 = copydata[:int(p*len(copydata))]
    list2 = copydata[int(p*len(copydata)):]
    #print('List 1: \n', list1)
    #print('\nList 2: \n',list2)
    
    return (list1, list2)

In [4]:
def norm(x):
    return np.sqrt(np.sum(x**2))

### Class dtree
Implementing the decision tree algorithm

In [5]:
class dtree:
    def __init__(self, n):
        self.k = n

### Working with Spotify Data

In [6]:
songs = pd.read_csv('Data/Spotify.csv', header=0)
print(f'Data Information:\nShape: {songs.shape}')
songs.head()

Data Information:
Shape: (1556, 23)


Index  Highest Charting Position  Number of Times Charted  \
0      1                          1                        8   
1      2                          2                        3   
2      3                          1                       11   
3      4                          3                        5   
4      5                          5                        1   

  Week of Highest Charting                          Song Name     Streams  \
0   2021-07-23--2021-07-30                            Beggin'  48,633,449   
1   2021-07-23--2021-07-30          STAY (with Justin Bieber)  47,248,719   
2   2021-06-25--2021-07-02                           good 4 u  40,162,559   
3   2021-07-02--2021-07-09                         Bad Habits  37,799,456   
4   2021-07-23--2021-07-30  INDUSTRY BABY (feat. Jack Harlow)  33,948,454   

           Artist Artist Followers                 Song ID  \
0        Måneskin          3377762  3Wrjm47oTz2sjIgck11l5e   
1   The Kid LAROI          2230022  5HCyWlXZPP0y6Gqq8TgA20   
2  Olivia Rodrigo          6266514  4ZtFanR9U6ndgddUvNcjcG   
3      Ed Sheeran         83293380  6PQ88X9TkUIAUIZJHW2upE   
4       Lil Nas X          5473565  27NovPIUIRrOZoCHxABJwK   

                                    Genre  ... Danceability Energy Loudness  \
0  ['indie rock italiano', 'italian pop']  ...        0.714    0.8   -4.808   
1                  ['australian hip hop']  ...        0.591  0.764   -5.484   
2                                 ['pop']  ...        0.563  0.664   -5.044   
3                       ['pop', 'uk pop']  ...        0.808  0.897   -3.712   
4           ['lgbtq+ hip hop', 'pop rap']  ...        0.736  0.704   -7.409   

  Speechiness Acousticness Liveness    Tempo Duration (ms) Valence  Chord  
0      0.0504        0.127    0.359  134.002        211560   0.589      B  
1      0.0483       0.0383    0.103  169.928        141806   0.478  C#/Db  
2       0.154        0.335   0.0849  166.928        178147   0.688      A  
3      0.0348       0.0469    0.364  126.026        231041   0.591      B  
4      0.0615       0.0203   0.0501  149.995        212000   0.894  D#/Eb  

[5 rows x 23 columns]

For knn, think of each genre as a class.

In [7]:
genres = songs['Genre']
print(genres)

0                  ['indie rock italiano', 'italian pop']
1                                  ['australian hip hop']
2                                                 ['pop']
3                                       ['pop', 'uk pop']
4                           ['lgbtq+ hip hop', 'pop rap']
                              ...                        
1551                       ['dance pop', 'pop', 'uk pop']
1552             ['sertanejo', 'sertanejo universitario']
1553    ['dance pop', 'electropop', 'pop', 'post-teen ...
1554                       ['brega funk', 'funk carioca']
1555                             ['pop', 'post-teen pop']
Name: Genre, Length: 1556, dtype: object


Need to determine the number of classes we have / how to classify songs with more than one genre:

Create a list of umbrella genres and fit each song into them. Then create a new column in the dataframe that holds the umbrella genre for each song (row).

In [8]:
umbrella_genres = ['indie rock', 'alternative', 'dance pop', 'latin', 'k-pop', 'reggaeton', 'alternative rock', 'comic', 'alt z',
                   'modern rock', 'melodic rap', 'folk', 'grunge', 'singer-songwriter', 'classic rock', 'edm', 'country',
                   'funk', 'pop rock', 'indie', 'metal', 'soul', 'trap', 'new wave', 'punk', 'boy band', 'swing', 'brooklyn drill',
                   'rock', 'hip hop', 'r&b', 'pop', 'rap', 'classical', 'psychedelic', 'Brazilian', 'easy listening', 'show tunes', 'jazz']

# create dictionary (in case the genres will need to be numbered): dict = { 1: 'indie rock', 2: 'alternative'}
dict = {}
count = 1
for u in umbrella_genres:
    dict[count] = u
    count += 1

print(f'Dictionary of Umbrella Genres: \n{dict}')

Dictionary of Umbrella Genres: 
{1: 'indie rock', 2: 'alternative', 3: 'dance pop', 4: 'latin', 5: 'k-pop', 6: 'reggaeton', 7: 'alternative rock', 8: 'comic', 9: 'alt z', 10: 'modern rock', 11: 'melodic rap', 12: 'folk', 13: 'grunge', 14: 'singer-songwriter', 15: 'classic rock', 16: 'edm', 17: 'country', 18: 'funk', 19: 'pop rock', 20: 'indie', 21: 'metal', 22: 'soul', 23: 'trap', 24: 'new wave', 25: 'punk', 26: 'boy band', 27: 'swing', 28: 'brooklyn drill', 29: 'rock', 30: 'hip hop', 31: 'r&b', 32: 'pop', 33: 'rap', 34: 'classical', 35: 'psychedelic', 36: 'Brazilian', 37: 'easy listening', 38: 'show tunes', 39: 'jazz'}


In [9]:
# assign each song to its umbrella genre
umbrella_songs = []

for g in genres:
    tracker = False # debugger: determining which songs are not getting an umbrella genre
    #print(f'g: {g}')
    
    # taking care of loads of random edge cases (songs with genres that are only held by very few instances)
    if g == "[]" or g == "" or g == " ":
        #print(f'EMPTY- ADDING: ?')
        umbrella_songs.append('?')
        tracker = True
    elif g.find("basshall") != -1 or g.find("tronica") != -1 or g.find("techno") != -1 or g.find("house") != -1:
        umbrella_songs.append("edm")
        tracker = True
    elif g.find("eurovision") != -1 or g.find("advocacy") != -1 or g.find("soundtrack") != -1:
        umbrella_songs.append("pop")
        tracker = True
    elif g.find("dream smp") != -1:
        umbrella_songs.append("classical")
        tracker = True
    elif g.find("weirdcore") != -1 or g.find("psych") != -1:
        umbrella_songs.append("psychedelic")
        tracker = True
    elif g.find("forro") != -1 or g.find("piseiro") != -1 or g.find("sertanejo") != -1:
        umbrella_songs.append("Brazilian")
        tracker = True 
    elif g.find("espanol") != -1 or g.find("mariachi") != -1 or g.find("mexican") != -1:
        umbrella_songs.append("latin")
        tracker = True  
    elif g.find("a cappella") != -1 or g.find("jawaiian") != -1:
        umbrella_songs.append("easy listening")
        tracker = True   
    else:
        # for each umbrella genre: if the umbrella genre is found in g, add it to the umbrella_songs
        for u in umbrella_genres:
            if g.find(u) != -1:
                #print(f'ADDING: {u}')
                umbrella_songs.append(u)
                tracker = True
                break # make sure only 1 u is being added to a g
    
    # tracking edge cases
    # if tracker == False:
    #     print(f'NOTHING ADDED for g: {g}')

In [10]:
# add umbrella genre column
songs = songs.assign(Umbrella_Genre = umbrella_songs)
songs.head()

Index  Highest Charting Position  Number of Times Charted  \
0      1                          1                        8   
1      2                          2                        3   
2      3                          1                       11   
3      4                          3                        5   
4      5                          5                        1   

  Week of Highest Charting                          Song Name     Streams  \
0   2021-07-23--2021-07-30                            Beggin'  48,633,449   
1   2021-07-23--2021-07-30          STAY (with Justin Bieber)  47,248,719   
2   2021-06-25--2021-07-02                           good 4 u  40,162,559   
3   2021-07-02--2021-07-09                         Bad Habits  37,799,456   
4   2021-07-23--2021-07-30  INDUSTRY BABY (feat. Jack Harlow)  33,948,454   

           Artist Artist Followers                 Song ID  \
0        Måneskin          3377762  3Wrjm47oTz2sjIgck11l5e   
1   The Kid LAROI          2230022  5HCyWlXZPP0y6Gqq8TgA20   
2  Olivia Rodrigo          6266514  4ZtFanR9U6ndgddUvNcjcG   
3      Ed Sheeran         83293380  6PQ88X9TkUIAUIZJHW2upE   
4       Lil Nas X          5473565  27NovPIUIRrOZoCHxABJwK   

                                    Genre  ... Energy Loudness Speechiness  \
0  ['indie rock italiano', 'italian pop']  ...    0.8   -4.808      0.0504   
1                  ['australian hip hop']  ...  0.764   -5.484      0.0483   
2                                 ['pop']  ...  0.664   -5.044       0.154   
3                       ['pop', 'uk pop']  ...  0.897   -3.712      0.0348   
4           ['lgbtq+ hip hop', 'pop rap']  ...  0.704   -7.409      0.0615   

  Acousticness Liveness    Tempo Duration (ms) Valence  Chord Umbrella_Genre  
0        0.127    0.359  134.002        211560   0.589      B     indie rock  
1       0.0383    0.103  169.928        141806   0.478  C#/Db        hip hop  
2        0.335   0.0849  166.928        178147   0.688      A            pop  
3       0.0469    0.364  126.026        231041   0.591      B            pop  
4       0.0203   0.0501  149.995        212000   0.894  D#/Eb        hip hop  

[5 rows x 24 columns]

Now lets try to use the knn algorithm to classify a song into one of the umbrella genres based on its duration and tempo.
I have a feeling these features will not be the best predictors, but we must start somewhere.

In [25]:
knn = knnalg(k)

In [42]:
# need to bring in the duration and tempo values as INTS not STRINGS
# ********* NOT WORKING *************

durations_strings = songs['Duration (ms)'].values
print(durations_strings)

durations = []
for d in durations_strings:
    d1 = int(d)
    durations.append(d1)

['211560' '141806' '178147' ... '217307' '152784' '221307']


ValueError: invalid literal for int() with base 10: ' '

In [31]:
# duration_tempo = [(songs['Duration (ms)'][i], songs['Tempo'][i]) for i in range(1556)]
# #print(duration_tempo)

# p = 0.6
# n = len(duration_tempo)
# rets = divideSets(p, duration_tempo)
# list1 = rets[0] 
# list2 = rets[1] 

# # need to get the data in one array & the labels in another
# data1 = [t[0] for t in list1]
# labels1 = [t[1] for t in list1]
# #print('\nData (List1):\n', data1)
# #print('\nLabels (List1):\n', labels1)

# data2 = [t[0] for t in list2]
# labels2 = [t[1] for t in list2]
# #print('\nData (List2):\n', data2)
# #print('\nLabels (List2):\n', labels2)

# k = 3
# knn.train(data1, labels1)
# predictions_labels = []
# for i in tqdm.tqdm(range(len(data2))):
#     predictions_labels.append(knn.predict(data2[i]))

# # want predictions with their points
# predictions = [ (data2[i], predictions_labels[i]) for i in range(len(predictions_labels))]
# #print(predictions)
# print(f'\np = {p:1.1}')
# print('Confusion Matrix: \n', confusion_matrix(labels2, predictions_labels))

  0%|          | 0/623 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for -: 'str' and 'str'